In [1]:
import numpy as np
import sys, os
from imageio.v2 import imread
import pywt
from tqdm import tqdm
from skimage.restoration import denoise_wavelet, estimate_sigma
from functools import partial
# rescale_sigma=True required to silence deprecation warnings
_denoise_wavelet = partial(denoise_wavelet, rescale_sigma=True)
import scipy.stats as stats
import cv2

In [2]:
# =========================================================
def rescale(dat,mn,mx):
    """
    rescales an input dat between mn and mx
    """
    m = min(dat.flatten())
    M = max(dat.flatten())
    return (mx-mn)*(dat-m)/(M-m)+mn

##====================================
def standardize(img):
    img = np.array(img)
    #standardization using adjusted standard deviation
    N = np.shape(img)[0] * np.shape(img)[1]
    s = np.maximum(np.std(img), 1.0/np.sqrt(N))
    m = np.mean(img)
    img = (img - m) / s
    img = rescale(img, 0, 1)
    del m, s, N

    return img

In [3]:
image= "C_4.jpg"

resolution = 0.088068182
img = cv2.imread(image)
nx, ny, _ = img.shape
width = max(nx, ny)
maxscale= width*resolution / 8

x= 0
verbose = 1

Stage 1: read image

In [4]:
im = imread(image)   # read the image straight with imread
im = np.squeeze(im)  # squeeze singleton dimensions
if len(np.shape(im))>3:
    im = im[:, :, :3]            # only keep the first 3 bands

if len(np.shape(im))==3: # if rgb, convert to grey
    im = (0.299 * im[:,:,0] + 0.5870*im[:,:,1] + 0.114*im[:,:,2]).astype('uint8')

nx,ny = np.shape(im)
if nx>ny:
    im=im.T

im = standardize(im)

Stage 2: Denoised image using default parameters of `denoise_wavelet`

In [5]:
filter=False

if filter:
    sigma_est = estimate_sigma(im, multichannel=False, average_sigmas=True)
    region = denoise_wavelet(im, multichannel=False, rescale_sigma=True,
                                method='VisuShrink', mode='soft', sigma=sigma_est*2)
else:
    region = im.copy()

original = rescale(region,0,255)

nx, ny = original.shape

Stage 3: Call cwt to get particle size distribution

In [13]:
P = []; M = []
for k in np.linspace(1,nx-1,100):
   [cfs, frequencies] = pywt.cwt(original[int(k),:], np.arange(3, np.maximum(nx,ny)/maxscale, 1),  'morl' , .5)
   period = 1. / frequencies
   power =(abs(cfs)) ** 2
   power = np.mean(np.abs(power), axis=1)/(period**2)
   P.append(power)

   M.append(period[np.argmax(power)])


In [14]:
print(np.shape(power))
print(np.shape(P))


(88,)
(100, 88)


In [15]:
p = np.mean(np.vstack(P), axis=0)

p = np.array(p/np.sum(p))

print(np.shape(p))
# get real scales by multiplying by resolution (mm/pixel)
scales = np.array(period)*resolution

srt = np.sqrt(np.sum(p*((scales-np.mean(M))**2)))

p = p+stats.norm.pdf(scales, np.mean(M), srt/2)
p = np.hstack([0,p])
scales = np.hstack([0,scales])
p = p/np.sum(p)

# area-by-number to volume-by-number
r_v = (p*scales**x) / np.sum(p*scales**x) #volume-by-weight proportion

(88,)


In [ ]:
h = 2
hs = 4

test1 = h * hs**2
print(test1)

Stage 5: Calc particle size stats

In [16]:

pd = np.interp([.05,.1,.16,.25,.3,.5,.75,.84,.9,.95],np.hstack((0,np.cumsum(r_v))), np.hstack((0,scales)) )
if verbose==1:
    print("d50 = "+str(pd[5]))

mnsz = np.sum(r_v*scales)
if verbose==1:
    print("mean size = "+str(mnsz))

srt = np.sqrt(np.sum(r_v*((scales-mnsz)**2)))
if verbose==1:
    print("stdev = "+str(srt))

sk = (sum(r_v*((scales-mnsz)**3)))/(100*srt**3)
if verbose==1:
    print("skewness = "+str(sk))

kurt = (sum(r_v*((scales-mnsz)**4)))/(100*srt**4)
if verbose==1:
    print("kurtosis = "+str(kurt))


d50 = 3.6155200895375654
mean size = 3.39645253008219
stdev = 1.1335927370694576
skewness = -0.008636888260900135
kurtosis = 0.030506515157315648


Stage 6: Return a dict object of stats

In [10]:
print( {'mean grain size': mnsz, 'grain size sorting': srt, 'grain size skewness': sk, 'grain size kurtosis': kurt, 
        'percentiles': [.05,.1,.16,.25,.3,.5,.75,.84,.9,.95], 'percentile_values': pd, 'grain size frequencies': r_v, 'grain size bins': scales})

{'mean grain size': 3.4078700058290634, 'grain size sorting': 1.1338662244142659, 'grain size skewness': -0.00867222956567473, 'grain size kurtosis': 0.030545330781198272, 'percentiles': [0.05, 0.1, 0.16, 0.25, 0.3, 0.5, 0.75, 0.84, 0.9, 0.95], 'percentile_values': array([1.00340185, 1.68202037, 2.20330136, 2.72222573, 2.94421878,
       3.62625094, 4.28488708, 4.50265626, 4.646084  , 4.76550945]), 'grain size frequencies': array([0.        , 0.00184878, 0.00254702, 0.00307818, 0.00342343,
       0.0035489 , 0.00356424, 0.00354501, 0.00350997, 0.0034819 ,
       0.00343186, 0.00339262, 0.00339267, 0.00340895, 0.00343373,
       0.00349421, 0.00356512, 0.00362729, 0.00373767, 0.00385809,
       0.00397635, 0.00410944, 0.00426132, 0.00443287, 0.00461466,
       0.00481741, 0.00504084, 0.00528045, 0.00553195, 0.00578554,
       0.00607421, 0.0063723 , 0.00668131, 0.00699674, 0.00733897,
       0.00768979, 0.00805175, 0.00843715, 0.00883586, 0.00924533,
       0.0096786 , 0.01011119, 0.010